# Clustering Metrics

This script evaluates clustering metrics for object detection results using the YOLO/RTDETR model. It computes a custom mAP@50 metric on clusters generated with DBSCAN.

In [ ]:
import os
from ultralytics import YOLO, RTDETR
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import squareform # Not directly used in DBSCAN with precomputed, but good to keep if needed elsewhere
import subprocess

In [41]:
# Image size (needed to denormalize/normalize YOLO boxes)
IMG_WIDTH = 320
IMG_HEIGHT = 320
DATASET_DIR = "../../datasets"

# --- Original function (for ground truth labels, no confidence) ---
def read_yolo_labels(file_path):
    """
    Reads a YOLO label file (ground truth) and returns list of (class_id, [x1, y1, x2, y2]) in absolute pixels.
    Format: class_id x_center y_center width height
    """
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            # Convert from YOLO normalized format to [x1, y1, x2, y2] absolute pixels
            x1 = (x_center - w / 2) * IMG_WIDTH
            y1 = (y_center - h / 2) * IMG_HEIGHT
            x2 = (x_center + w / 2) * IMG_WIDTH
            y2 = (y_center + h / 2) * IMG_HEIGHT
            boxes.append((class_id, [x1, y1, x2, y2]))
    return boxes

# --- NEW function to read prediction files (includes confidence) ---
def read_yolo_predictions(file_path):
    """
    Reads a YOLO prediction file and returns list of (class_id, confidence, [x1, y1, x2, y2]) in absolute pixels.
    Format: class_id confidence x_center y_center width height
    """
    predictions = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6: # Ensure enough parts for class, conf, and bbox
                print(f"Warning: Skipping malformed line in {file_path}: {line.strip()}")
                continue
            class_id = int(parts[0])
            confidence = float(parts[5])
            x_center, y_center, w, h = map(float, parts[1:5])
            # Convert from YOLO normalized format to [x1, y1, x2, y2] absolute pixels
            x1 = (x_center - w / 2) * IMG_WIDTH
            y1 = (y_center - h / 2) * IMG_HEIGHT
            x2 = (x_center + w / 2) * IMG_WIDTH
            y2 = (y_center + h / 2) * IMG_HEIGHT
            predictions.append((class_id, [x1, y1, x2, y2], confidence))
    return predictions

def get_centroid(box_coords):
    """Calculates the centroid of a bounding box [x1, y1, x2, y2]."""
    x1, y1, x2, y2 = box_coords
    return [(x1 + x2) / 2, (y1 + y2) / 2]

# --- Modified function to get enclosing box AND mean confidence ---
def get_enclosing_box_and_conf(cluster_entries):
    """
    Calculates the enclosing bounding box and mean confidence for a cluster.
    cluster_entries: list of (box_coords, confidence) tuples, where box_coords is [x1, y1, x2, y2].
    Returns: [min_x1, min_y1, max_x2, max_y2, mean_conf]
    """
    if not cluster_entries:
        return None

    x1s, y1s, x2s, y2s, confs = [], [], [], [], []
    for box_coords, conf in cluster_entries:
        x1, y1, x2, y2 = box_coords
        x1s.append(x1)
        y1s.append(y1)
        x2s.append(x2)
        y2s.append(y2)
        if conf is not None: # Only append if confidence exists
            confs.append(conf)

    min_x1 = min(x1s)
    min_y1 = min(y1s)
    max_x2 = max(x2s)
    max_y2 = max(y2s)

    # Calculate mean confidence, default to 1.0 if no confs or empty
    mean_conf = np.mean(confs) if confs else 1.0

    return [min_x1, min_y1, max_x2, max_y2, mean_conf]

def compute_custom_dist_matrix(boxes_coords_only):
    """
    Computes a custom distance matrix between bounding boxes based on overlap.
    boxes_coords_only: list of [x1, y1, x2, y2]
    """
    n = len(boxes_coords_only)
    dist_matrix = np.zeros((n, n))

    for i in range(n):
        x1_c, y1_c = get_centroid(boxes_coords_only[i])
        w1 = boxes_coords_only[i][2] - boxes_coords_only[i][0]
        h1 = boxes_coords_only[i][3] - boxes_coords_only[i][1]
        for j in range(i + 1, n):
            x2_c, y2_c = get_centroid(boxes_coords_only[j])
            w2 = boxes_coords_only[j][2] - boxes_coords_only[j][0]
            h2 = boxes_coords_only[j][3] - boxes_coords_only[j][1]

            # This distance metric seems to be based on non-overlapping distance
            # If dx or dy are negative, it means there's overlap in that dimension.
            # max(0, dx, dy) ensures distance is 0 if boxes overlap or touch.
            dx = abs(x1_c - x2_c) - (w1 + w2) / 2
            dy = abs(y1_c - y2_c) - (h1 + h2) / 2
            distance_x = max(0, dx)
            distance_y = max(0, dy) # This is a custom distance, not IoU based
            distance = (distance_x * distance_x + distance_y * distance_y) ** 0.5  # Euclidean distance

            dist_matrix[i, j] = dist_matrix[j, i] = distance
    return dist_matrix

# --- Modified clustering function to handle confidence and be flexible for GT/Predictions ---
def cluster_boxes_per_image(file_path, eps=1, min_samples=1, is_prediction=False):
    """
    Clusters bounding boxes within a single image file based on a custom distance metric using DBSCAN.
    Can process either ground truth labels (no confidence) or predictions (with confidence).

    Args:
        file_path (str): Path to the YOLO label/prediction file.
        eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        min_samples (int): The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
        is_prediction (bool): If True, reads file as predictions (with confidence). Otherwise, as ground truth.

    Returns:
        dict: A dictionary where keys are class_ids and values are lists of
              [x1, y1, x2, y2, mean_confidence] for each cluster.
    """
    class_to_entries = {} # Stores (class_id, (box_coords, confidence))

    if is_prediction:
        entries = read_yolo_predictions(file_path)
        for class_id, box_coords, conf in entries:
            class_to_entries.setdefault(class_id, []).append((box_coords, conf))
    else:
        entries = read_yolo_labels(file_path)
        for class_id, box_coords in entries:
            class_to_entries.setdefault(class_id, []).append((box_coords, 1.0)) # Default confidence for GT

    clusters_per_class = {}

    for class_id, entries_for_class in class_to_entries.items():
        if not entries_for_class: # Skip if no boxes for this class
            continue
        # Separate box coordinates for distance calculation
        boxes_coords_only = [entry[0] for entry in entries_for_class]
        
        # Compute distance matrix only on coordinates
        dist_matrix = compute_custom_dist_matrix(boxes_coords_only)
        
        # DBSCAN expects a 2D array, even for a single sample (though min_samples=1 handles this)
        if len(boxes_coords_only) == 1:
            # DBSCAN with precomputed metric needs a 1x1 matrix for a single point
            # If min_sample is 1, a single point is a cluster.
            labels = np.array([0]) # Assign to cluster 0
        else:
            db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
            labels = db.fit_predict(dist_matrix)

        clusters_raw = {} # Stores raw entries (box_coords, conf) for each cluster label
        for lbl, entry in zip(labels, entries_for_class):
            if lbl == -1: # Noise points are ignored as per original code
                continue
            clusters_raw.setdefault(lbl, []).append(entry)

        # Process raw clusters to get enclosing box and mean confidence
        clusters_per_class[class_id] = [get_enclosing_box_and_conf(c) for c in clusters_raw.values()]

    return clusters_per_class

# --- Modified save_clusters function ---
def save_clusters(src_dir, save_cluster_labels_dir, is_prediction_data=False):
    """
    Processes files in src_dir, clusters bounding boxes, and saves the results.
    For prediction data, it includes the mean confidence of the cluster.

    Args:
        src_dir (str): Directory containing the source YOLO label/prediction files.
        save_cluster_labels_dir (str): Directory where clustered label files will be saved.
        is_prediction_data (bool): Set to True if src_dir contains prediction files
                                   (which have confidence scores).
    """
    set_clusters = {}
    print(f"Processing files in {src_dir}...")
    for fname in sorted(os.listdir(src_dir)):
        if not fname.endswith('.txt'):
            continue
        path = os.path.join(src_dir, fname)
        # Pass the is_prediction flag to the clustering function
        clusters = cluster_boxes_per_image(path, eps=50, min_samples=1, is_prediction=is_prediction_data)
        set_clusters[fname] = clusters

    # # Print results (optional, for debugging/overview)
    # print("\n--- Clustering Results Summary ---")
    # for fname, clusters in set_clusters.items():
    #     print(f"File: {fname}")
    #     if not clusters:
    #         print("  No clusters found.")
    #         continue
    #     for class_id, boxes_with_conf in clusters.items():
    #         print(f"  Class ID: {class_id}")
    #         for box in boxes_with_conf:
    #             # box is [x1, y1, x2, y2, mean_conf]
    #             print(f"    Enclosing Box: [{box[0]:.2f}, {box[1]:.2f}, {box[2]:.2f}, {box[3]:.2f}], Mean Conf: {box[4]:.4f}")
    #     print()

    save_dir = save_cluster_labels_dir
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    print(f"Saving clustered results to {save_dir}...")

    # Save results with format of YOLO label files (class_id x_center y_center width height [confidence])
    for fname, clusters in set_clusters.items():
        output_path = os.path.join(save_dir, fname)
        with open(output_path, 'w') as f:
            # Optional: print if multiple classes for a file (as in your original code)
            # This logic is a bit odd as it prints to console, not file, and only if >1 class
            # if len(clusters) > 1:
            #     print(f"Multiple classes found in {fname}, writing to file.")
            #     for class_id, boxes in clusters.items():
            #         print(f"  Class ID: {class_id}, Enclosing Boxes: {len(boxes)}")
                        
            for class_id, boxes_with_conf in clusters.items():
                for box_data in boxes_with_conf:
                    # box_data is [x1, y1, x2, y2, mean_conf]
                    x1, y1, x2, y2, mean_conf = box_data
                    
                    x_center = (x1 + x2) / 2 / IMG_WIDTH
                    y_center = (y1 + y2) / 2 / IMG_HEIGHT
                    width = (x2 - x1) / IMG_WIDTH
                    height = (y2 - y1) / IMG_HEIGHT
                    
                    # Write class_id x_center y_center width height confidence
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {mean_conf:.6f}\n")

In [42]:
def yolo_to_xyxy(box, img_w, img_h):
    cls, x_c, y_c, w, h = box[:5]
    x1 = (x_c - w / 2) * img_w
    y1 = (y_c - h / 2) * img_h
    x2 = (x_c + w / 2) * img_w
    y2 = (y_c + h / 2) * img_h
    return [int(cls), x1, y1, x2, y2]

def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

def ap_per_class(tp, conf, pred_cls, target_cls):
    indices = np.argsort(-conf)
    tp, pred_cls = tp[indices], pred_cls[indices]
    unique_classes = np.unique(np.concatenate((pred_cls, target_cls)))
    ap = []

    for c in unique_classes:
        idx = pred_cls == c
        n_gt = (target_cls == c).sum()
        n_pred = idx.sum()

        if n_gt == 0 or n_pred == 0:
            ap.append(0)
            continue

        fpc = (1 - tp[idx]).cumsum()
        tpc = tp[idx].cumsum()
        recall = tpc / (n_gt + 1e-16)
        precision = tpc / (tpc + fpc + 1e-16)

        ap_c = np.trapezoid(precision, recall)
        ap.append(ap_c)

    return np.mean(ap)

def load_labels(file_path, img_w, img_h):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            boxes.append(yolo_to_xyxy(vals, img_w, img_h))
    return boxes

def load_predictions(file_path, img_w, img_h):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            cls, x1, y1, x2, y2 = yolo_to_xyxy(vals[:5], img_w, img_h)
            conf = vals[5]
            boxes.append([cls, x1, y1, x2, y2, conf])
    return boxes

def compute_map_from_folders(img_dir, label_dir, pred_dir, iou_thr=0.5):
    tp, confs, pred_cls, target_cls = [], [], [], []

    for img_file in tqdm(os.listdir(img_dir)):
        if not img_file.endswith('.jpg'):
            continue

        base = os.path.splitext(img_file)[0]
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, base + '.txt')
        pred_path = os.path.join(pred_dir, base + '.txt')

        img = Image.open(img_path)
        w, h = img.size

        if not os.path.exists(label_path):
            continue
        gt = load_labels(label_path, w, h)
        preds = load_predictions(pred_path, w, h) if os.path.exists(pred_path) else []

        matched = []
        for pred in preds:
            cls_p, x1p, y1p, x2p, y2p, conf = pred
            best_iou = 0
            best_idx = -1
            for i, gt_box in enumerate(gt):
                cls_g, x1g, y1g, x2g, y2g = gt_box
                if cls_p != cls_g or i in matched:
                    continue
                iou = compute_iou([x1p, y1p, x2p, y2p], [x1g, y1g, x2g, y2g])
                if iou > best_iou:
                    best_iou = iou
                    best_idx = i

            if best_iou >= iou_thr:
                tp.append(1)
                matched.append(best_idx)
            else:
                tp.append(0)
            confs.append(conf)
            pred_cls.append(cls_p)

        for gt_box in gt:
            target_cls.append(gt_box[0])

    map50 = ap_per_class(np.array(tp), np.array(confs), np.array(pred_cls), np.array(target_cls))
    return map50

In [ ]:
def predict_cluster_and_compute_map(model_weights_path, dataset_name, fold_pred_dir):
    """
    Predicts using the model weights and computes mAP@0.5 for the specified dataset.
    """
    # Print all arguments
    print(f"Predicting with model weights: {model_weights_path}")
    print(f"Dataset name: {dataset_name}")
    print(f"Fold prediction directory: {fold_pred_dir}")
    model = YOLO(model_weights_path) if "yolo" in model_weights_path else RTDETR(model_weights_path)

    # mantain old predictions if they exist and are the same len of images
    if os.path.exists(os.path.join(fold_pred_dir, "labels_clusters")):
        #subprocess.run(["rm", "-rf", fold_pred_dir], check=True)
        #print(f"Removed old predictions in {fold_pred_dir}")
        # print that already exists: skip the prediction
        print(f"Predictions already exist in {fold_pred_dir}, skipping prediction.")
    else:
        # remove old predictions
        subprocess.run(["rm", "-rf", fold_pred_dir], check=True)
        print(f"Removed old predictions in {fold_pred_dir}")

        print(f"Saving predictions to {fold_pred_dir}...")
        # Predict
        model.predict(
            source=os.path.join(DATASET_DIR, dataset_name, "test", "images"),
            save=True,
            save_txt=True,
            save_conf=True,
            # project is the parent directory of fold_pred_dir
            project= os.path.dirname(fold_pred_dir),
            name= os.path.basename(fold_pred_dir),
            conf=0.25,  # Confidence threshold
            iou=0.45,   # IoU threshold
            device="cuda" if os.path.exists("/dev/nvidia0") else "cpu",
        )

    # Save clusters for the predicted labels
    save_clusters(
        os.path.join(fold_pred_dir, "labels"),
        os.path.join(fold_pred_dir, "labels_clusters"),
        is_prediction_data=True
    )
    
    # Compute mAP@0.5
    map50 = compute_map_from_folders(
        os.path.join(DATASET_DIR, dataset_name, "test", "images"),
        os.path.join(DATASET_DIR, dataset_name, "test", "labels_clusters"),
        os.path.join(fold_pred_dir, "labels_clusters"),
        iou_thr=0.5
    )

    print(f"mAP@0.5 for {fold_pred_dir}: {map50:.4f}")
    
    return map50


def compute_map50_of_model(model_dir, dataset_name):
    # compute mAP@50 for each fold
    model_name = os.path.basename(model_dir)
    folds_dirs = [d for d in os.listdir(model_dir) if "fold" in d and os.path.isdir(os.path.join(model_dir, d))]
    map50s_folds = []
    for fold_dir in folds_dirs:
        fold_path = os.path.join(model_dir, fold_dir)
        fold_pred_dir = os.path.join(fold_path, "predict")
        map50 = predict_cluster_and_compute_map(
            model_weights_path=os.path.join(fold_path, "weights", "best.pt"),
            dataset_name=dataset_name,
            fold_pred_dir=fold_pred_dir
        )
        map50s_folds.append(map50)

    # Compute mean and std of mAP@50 across folds
    if map50s_folds:
        mean_map50 = np.mean(map50s_folds)
        std_map50 = np.std(map50s_folds)
        print(f"Model: {model_name}, mAP@0.5: {mean_map50:.4f} ± {std_map50:.4f}")

    return {
        "mean": mean_map50,
        "std": std_map50,
    }

def compute_map50_of_all_models(runs_dir, dataset_name):
    """
    Computes mAP@0.5 for all models in the specified runs directory for the given dataset.
    """
    model_dirs = [d for d in os.listdir(runs_dir) if os.path.isdir(os.path.join(runs_dir, d))]
    all_map50s = {}
    
    for model_dir in model_dirs:
        model_path = os.path.join(runs_dir, model_dir)
        map50s = compute_map50_of_model(model_path, dataset_name)
        all_map50s[model_dir] = map50s
    
    return all_map50s

def compute_and_print_map50(runs_dir, dataset_name):
    """
    Computes and prints mAP@0.5 for all models in the specified runs directory for the given dataset.
    """
    # generate clusters for the dataset labels
    dataset_dir = os.path.join(DATASET_DIR, dataset_name)
    save_clusters(
        os.path.join(dataset_dir, "test", "labels"),
        os.path.join(dataset_dir, "test", "labels_clusters"),
        is_prediction_data=False
    )

    all_map50s = compute_map50_of_all_models(runs_dir, dataset_name)
    
    print("\n--- Summary of mAP@0.5 for all models ---")
    for model, map50s in all_map50s.items():
        print(f"Model: {model}")
        for fold, map50 in map50s.items():
            if isinstance(map50, dict):  # This is the mean/std entry
                print(f"  {fold}: {map50['mean']:.4f} ± {map50['std']:.4f}")
            else:
                print(f"  {fold}: {map50:.4f}")

    return all_map50s

# Usage
all_maps = compute_and_print_map50(
    runs_dir="../../runs",
    dataset_name="roboflow"
)

Processing files in ../../datasets/roboflow/test/labels...
Saving clustered results to ../../datasets/roboflow/test/labels_clusters...
Predicting with model weights: ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/weights/best.pt
Dataset name: roboflow
Fold prediction directory: ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/predict
Predictions already exist in ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/predict, skipping prediction.
Processing files in ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/predict/labels...
Saving clustered results to ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/predict/labels_clusters...


100%|██████████| 1099/1099 [00:00<00:00, 8266.60it/s]

mAP@0.5 for ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_0/predict: 0.8702
Predicting with model weights: ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_1/weights/best.pt
Dataset name: roboflow
Fold prediction directory: ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_1/predict


Predictions already exist in ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_1/predict, skipping prediction.
Processing files in ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_1/predict/labels...
Saving clustered results to ../../runs/stage2_yolo11m_k_fold_cv_augmented/yolo11m_fold_1/predict/labels_clusters...


100%|██████████| 1099/1099 [00:00<00:00, 8478.67it/s]

...

100%|██████████| 1099/1099 [00:00<00:00, 8321.33it/s]

mAP@0.5 for ../../runs/stage2_yolo11s_k_fold_cv_augmented/yolo11s_fold_0/predict: 0.8591
Model: stage2_yolo11s_k_fold_cv_augmented, mAP@0.5: 0.8538 ± 0.0044
Predicting with model weights: ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/weights/best.pt
Dataset name: roboflow
Fold prediction directory: ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/predict


Predictions already exist in ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/predict, skipping prediction.
Processing files in ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/predict/labels...
Saving clustered results to ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/predict/labels_clusters...


100%|██████████| 1099/1099 [00:00<00:00, 8466.93it/s]

mAP@0.5 for ../../runs/stage3_yolo11x_fold0_subsampled/yolo11x_fold_0/predict: 0.8550
Model: stage3_yolo11x_fold0_subsampled, mAP@0.5: 0.8550 ± 0.0000

--- Summary of mAP@0.5 for all models ---
Model: stage2_yolo11m_k_fold_cv_augmented
  mean: 0.8587
  std: 0.0098
Model: stage3_yolo11l_fold0_subsampled
  mean: 0.8562
  std: 0.0000
Model: stage1_yolo11n_k_fold_cv
  mean: 0.8508
  std: 0.0106
Model: stage2_yolo11n_k_fold_cv_augmented
  mean: 0.8584
  std: 0.0053
Model: stage4_rtdetr-l_fold0_augmented
  mean: 0.8268
  std: 0.0000
Model: stage4_rtdetr-x_fold0_augmented
  mean: 0.8151
  std: 0.0000
Model: stage2_yolo11s_k_fold_cv_augmented
  mean: 0.8538
  std: 0.0044
Model: stage3_yolo11x_fold0_subsampled
  mean: 0.8550
  std: 0.0000


In [50]:
# Print the values of mean and std for mAP@50, ordering in this way:
# yolo{n,s,m,l,x}, rt-detr{l,x}
# using 3 decimal places
print("mAP@50 values for all models:")

order = [
    "stage1_yolo11n",
    "stage2_yolo11n",
    "yolo11s",
    "yolo11m",
    "yolo11l",
    "yolo11x",
    "rtdetr-l",
    "rtdetr-x",
]

for model in order:
    # for all keys check if it contains the model name
    for key, value in all_maps.items():
        if model in key:
            mean = value.get("mean", 0)
            std = value.get("std", 0)
            print(f"{model}: {mean:.3f}$\pm${std:.3f}")
            break
    else:
        print(f"{model}: Not found in results")

mAP@50 values for all models:
stage1_yolo11n: 0.851$\pm$0.011
stage2_yolo11n: 0.858$\pm$0.005
yolo11s: 0.854$\pm$0.004
yolo11m: 0.859$\pm$0.010
yolo11l: 0.856$\pm$0.000
yolo11x: 0.855$\pm$0.000
rtdetr-l: 0.827$\pm$0.000
rtdetr-x: 0.815$\pm$0.000
